In [126]:
import os
import json
import random
import sqlite3
import sqlparse
import pandas as pd
import numpy as np

import seaborn as sns
import mysql.connector

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import PolynomialFeatures

import matplotlib.pyplot as plt

In [127]:
import pandas as pd # Importing the pandas library
import numpy as np

# prompt: upload rthe csv

#from google.colab import files
#uploaded = files.upload()
# Assuming the uploaded file is named 'your_file.csv'
# Replace 'your_file.csv' with the actual file name if it's different
df = pd.read_csv('Airbnb_Open_Data.csv') # Now pd is recognized
df


C:\Users\bonas\AppData\Local\Temp\ipykernel_23460\2142447298.py:10: DtypeWarning: Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('Airbnb_Open_Data.csv') # Now pd is recognized


,id,NAME,host id,host_identity_verified,host name,neighbourhood group,neighbourhood,lat,long,country,...,service fee,minimum nights,number of reviews,last review,reviews per month,review rate number,calculated host listings count,availability 365,house_rules,license
0,1001254,Clean & quiet apt home by the park,80014485718,unconfirmed,Madaline,Brooklyn,Kensington,40.64749,-73.97237,United States,...,$193,10.0,9.0,10/19/2021,0.21,4.0,6.0,286.0,Clean up and treat the home the way you'd like...,NaN
1,1002102,Skylit Midtown Castle,52335172823,verified,Jenna,Manhattan,Midtown,40.75362,-73.98377,United States,...,$28,30.0,45.0,5/21/2022,0.38,4.0,2.0,228.0,Pet friendly but please confirm with me if the...,NaN
2,1002403,THE VILLAGE OF HARLEM....NEW YORK !,78829239556,NaN,Elise,Manhattan,Harlem,40.80902,-73.94190,United States,...,$124,3.0,0.0,NaN,NaN,5.0,1.0,352.0,"I encourage you to use my kitchen, cooking and...",NaN
3,1002755,NaN,85098326012,unconfirmed,Garry,Brooklyn,Clinton Hill,40.68514,-73.95976,United States,...,$74,30.0,270.0,7/5/2019,4.64,4.0,1.0,322.0,NaN,NaN
4,1003689,Entire Apt: Spacious Studio/Loft by central park,92037596077,verified,Lyndon,Manhattan,East Harlem,40.79851,-73.94399,United States,...,$41,10.0,9.0,11/19/2018,0.10,3.0,1.0,289.0,"Please no smoking in the house, porch or on th...",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102594,6092437,Spare room in Williamsburg,12312296767,verified,Krik,Brooklyn,Williamsburg,40.70862,-73.94651,United States,...,$169,1.0,0.0,NaN,NaN,3.0,1.0,227.0,No Smoking No Parties or Events of any kind Pl...,NaN
102595,6092990,Best Location near Columbia U,77864383453,unconfirmed,Mifan,Manhattan,Morningside Heights,40.80460,-73.96545,United States,...,$167,1.0,1.0,7/6/2015,0.02,2.0,2.0,395.0,House rules: Guests agree to the following ter...,NaN
102596,6093542,"Comfy, bright room in Brooklyn",69050334417,unconfirmed,Megan,Brooklyn,Park Slope,40.67505,-73.98045,United States,...,$198,3.0,0.0,NaN,NaN,5.0,1.0,342.0,NaN,NaN
102597,6094094,Big Studio-One Stop from Midtown,11160591270,unconfirmed,Christopher,Queens,Long Island City,40.74989,-73.93777,United States,...,$109,2.0,5.0,10/11/2015,0.10,3.0,1.0,386.0,NaN,NaN


In [128]:
# df = pd.read_csv('/kaggle/input/airbnbopendata/Airbnb_Open_Data.csv')

In [129]:
df.head()

,id,NAME,host id,host_identity_verified,host name,neighbourhood group,neighbourhood,lat,long,country,...,service fee,minimum nights,number of reviews,last review,reviews per month,review rate number,calculated host listings count,availability 365,house_rules,license
0,1001254,Clean & quiet apt home by the park,80014485718,unconfirmed,Madaline,Brooklyn,Kensington,40.64749,-73.97237,United States,...,$193,10.0,9.0,10/19/2021,0.21,4.0,6.0,286.0,Clean up and treat the home the way you'd like...,NaN
1,1002102,Skylit Midtown Castle,52335172823,verified,Jenna,Manhattan,Midtown,40.75362,-73.98377,United States,...,$28,30.0,45.0,5/21/2022,0.38,4.0,2.0,228.0,Pet friendly but please confirm with me if the...,NaN
2,1002403,THE VILLAGE OF HARLEM....NEW YORK !,78829239556,NaN,Elise,Manhattan,Harlem,40.80902,-73.94190,United States,...,$124,3.0,0.0,NaN,NaN,5.0,1.0,352.0,"I encourage you to use my kitchen, cooking and...",NaN
3,1002755,NaN,85098326012,unconfirmed,Garry,Brooklyn,Clinton Hill,40.68514,-73.95976,United States,...,$74,30.0,270.0,7/5/2019,4.64,4.0,1.0,322.0,NaN,NaN
4,1003689,Entire Apt: Spacious Studio/Loft by central park,92037596077,verified,Lyndon,Manhattan,East Harlem,40.79851,-73.94399,United States,...,$41,10.0,9.0,11/19/2018,0.10,3.0,1.0,289.0,"Please no smoking in the house, porch or on th...",NaN


# *Renaming the columns*

In [130]:
df.columns = df.columns.str.upper()

In [131]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102599 entries, 0 to 102598
Data columns (total 26 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   ID                              102599 non-null  int64  
 1   NAME                            102349 non-null  object 
 2   HOST ID                         102599 non-null  int64  
 3   HOST_IDENTITY_VERIFIED          102310 non-null  object 
 4   HOST NAME                       102193 non-null  object 
 5   NEIGHBOURHOOD GROUP             102570 non-null  object 
 6   NEIGHBOURHOOD                   102583 non-null  object 
 7   LAT                             102591 non-null  float64
 8   LONG                            102591 non-null  float64
 9   COUNTRY                         102067 non-null  object 
 10  COUNTRY CODE                    102468 non-null  object 
 11  INSTANT_BOOKABLE                102494 non-null  object 
 12  CANCELLATION_POL

In [132]:
df.columns

Index(['ID', 'NAME', 'HOST ID', 'HOST_IDENTITY_VERIFIED', 'HOST NAME',
       'NEIGHBOURHOOD GROUP', 'NEIGHBOURHOOD', 'LAT', 'LONG', 'COUNTRY',
       'COUNTRY CODE', 'INSTANT_BOOKABLE', 'CANCELLATION_POLICY', 'ROOM TYPE',
       'CONSTRUCTION YEAR', 'PRICE', 'SERVICE FEE', 'MINIMUM NIGHTS',
       'NUMBER OF REVIEWS', 'LAST REVIEW', 'REVIEWS PER MONTH',
       'REVIEW RATE NUMBER', 'CALCULATED HOST LISTINGS COUNT',
       'AVAILABILITY 365', 'HOUSE_RULES', 'LICENSE'],
      dtype='object')

# *Deleting the reduntant columns*

In [133]:
columns_to_keep = ['ID', 'NAME', 'HOST ID', 'HOST_IDENTITY_VERIFIED', 'HOST NAME',
       'NEIGHBOURHOOD GROUP', 'NEIGHBOURHOOD', 'LAT', 'LONG', 'COUNTRY',
       'COUNTRY CODE', 'INSTANT_BOOKABLE', 'CANCELLATION_POLICY', 'ROOM TYPE',
       'CONSTRUCTION YEAR', 'PRICE', 'SERVICE FEE', 'MINIMUM NIGHTS',
       'NUMBER OF REVIEWS','REVIEW RATE NUMBER', 'CALCULATED HOST LISTINGS COUNT','AVAILABILITY 365']

columns_to_drop = ['LICENSE','HOUSE_RULES','LAST REVIEW','REVIEWS PER MONTH']

In [134]:
df.drop(columns=columns_to_drop,inplace=True)

In [135]:
df.columns == columns_to_keep

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True])

In [136]:
df.head()

,ID,NAME,HOST ID,HOST_IDENTITY_VERIFIED,HOST NAME,NEIGHBOURHOOD GROUP,NEIGHBOURHOOD,LAT,LONG,COUNTRY,...,CANCELLATION_POLICY,ROOM TYPE,CONSTRUCTION YEAR,PRICE,SERVICE FEE,MINIMUM NIGHTS,NUMBER OF REVIEWS,REVIEW RATE NUMBER,CALCULATED HOST LISTINGS COUNT,AVAILABILITY 365
0,1001254,Clean & quiet apt home by the park,80014485718,unconfirmed,Madaline,Brooklyn,Kensington,40.64749,-73.97237,United States,...,strict,Private room,2020.0,$966,$193,10.0,9.0,4.0,6.0,286.0
1,1002102,Skylit Midtown Castle,52335172823,verified,Jenna,Manhattan,Midtown,40.75362,-73.98377,United States,...,moderate,Entire home/apt,2007.0,$142,$28,30.0,45.0,4.0,2.0,228.0
2,1002403,THE VILLAGE OF HARLEM....NEW YORK !,78829239556,NaN,Elise,Manhattan,Harlem,40.80902,-73.94190,United States,...,flexible,Private room,2005.0,$620,$124,3.0,0.0,5.0,1.0,352.0
3,1002755,NaN,85098326012,unconfirmed,Garry,Brooklyn,Clinton Hill,40.68514,-73.95976,United States,...,moderate,Entire home/apt,2005.0,$368,$74,30.0,270.0,4.0,1.0,322.0
4,1003689,Entire Apt: Spacious Studio/Loft by central park,92037596077,verified,Lyndon,Manhattan,East Harlem,40.79851,-73.94399,United States,...,moderate,Entire home/apt,2009.0,$204,$41,10.0,9.0,3.0,1.0,289.0


# *Dropping the duplicates*

In [137]:
df.duplicated().sum()

541

In [138]:
df.drop_duplicates(inplace=True)

In [139]:
df.duplicated().sum()

0

# *Removing the NaN values*

In [140]:
df.isna().sum()

ID                                  0
NAME                              250
HOST ID                             0
HOST_IDENTITY_VERIFIED            289
HOST NAME                         404
NEIGHBOURHOOD GROUP                29
NEIGHBOURHOOD                      16
LAT                                 8
LONG                                8
COUNTRY                           532
COUNTRY CODE                      131
INSTANT_BOOKABLE                  105
CANCELLATION_POLICY                76
ROOM TYPE                           0
CONSTRUCTION YEAR                 214
PRICE                             247
SERVICE FEE                       273
MINIMUM NIGHTS                    400
NUMBER OF REVIEWS                 183
REVIEW RATE NUMBER                319
CALCULATED HOST LISTINGS COUNT    319
AVAILABILITY 365                  448
dtype: int64

In [141]:
df.dropna(inplace=True)

In [142]:
df.isna().sum()

ID                                0
NAME                              0
HOST ID                           0
HOST_IDENTITY_VERIFIED            0
HOST NAME                         0
NEIGHBOURHOOD GROUP               0
NEIGHBOURHOOD                     0
LAT                               0
LONG                              0
COUNTRY                           0
COUNTRY CODE                      0
INSTANT_BOOKABLE                  0
CANCELLATION_POLICY               0
ROOM TYPE                         0
CONSTRUCTION YEAR                 0
PRICE                             0
SERVICE FEE                       0
MINIMUM NIGHTS                    0
NUMBER OF REVIEWS                 0
REVIEW RATE NUMBER                0
CALCULATED HOST LISTINGS COUNT    0
AVAILABILITY 365                  0
dtype: int64

# *Cleaning individual columns*

In [143]:
df.head(3)

,ID,NAME,HOST ID,HOST_IDENTITY_VERIFIED,HOST NAME,NEIGHBOURHOOD GROUP,NEIGHBOURHOOD,LAT,LONG,COUNTRY,...,CANCELLATION_POLICY,ROOM TYPE,CONSTRUCTION YEAR,PRICE,SERVICE FEE,MINIMUM NIGHTS,NUMBER OF REVIEWS,REVIEW RATE NUMBER,CALCULATED HOST LISTINGS COUNT,AVAILABILITY 365
0,1001254,Clean & quiet apt home by the park,80014485718,unconfirmed,Madaline,Brooklyn,Kensington,40.64749,-73.97237,United States,...,strict,Private room,2020.0,$966,$193,10.0,9.0,4.0,6.0,286.0
1,1002102,Skylit Midtown Castle,52335172823,verified,Jenna,Manhattan,Midtown,40.75362,-73.98377,United States,...,moderate,Entire home/apt,2007.0,$142,$28,30.0,45.0,4.0,2.0,228.0
4,1003689,Entire Apt: Spacious Studio/Loft by central park,92037596077,verified,Lyndon,Manhattan,East Harlem,40.79851,-73.94399,United States,...,moderate,Entire home/apt,2009.0,$204,$41,10.0,9.0,3.0,1.0,289.0


In [144]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 98509 entries, 0 to 102040
Data columns (total 22 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   ID                              98509 non-null  int64  
 1   NAME                            98509 non-null  object 
 2   HOST ID                         98509 non-null  int64  
 3   HOST_IDENTITY_VERIFIED          98509 non-null  object 
 4   HOST NAME                       98509 non-null  object 
 5   NEIGHBOURHOOD GROUP             98509 non-null  object 
 6   NEIGHBOURHOOD                   98509 non-null  object 
 7   LAT                             98509 non-null  float64
 8   LONG                            98509 non-null  float64
 9   COUNTRY                         98509 non-null  object 
 10  COUNTRY CODE                    98509 non-null  object 
 11  INSTANT_BOOKABLE                98509 non-null  object 
 12  CANCELLATION_POLICY             9850

# SERVICE FEE

In [145]:
df['SERVICE FEE']

0         $193 
1          $28 
4          $41 
5         $115 
7         $212 
          ...  
102029    $164 
102030     $91 
102031    $216 
102032     $21 
102040    $196 
Name: SERVICE FEE, Length: 98509, dtype: object

In [146]:
df['SERVICE FEE'].unique()

array(['$193 ', '$28 ', '$41 ', '$115 ', '$212 ', '$204 ', '$58 ', '$64 ',
       '$121 ', '$116 ', '$30 ', '$219 ', '$113 ', '$74 ', '$171 ',
       '$118 ', '$168 ', '$106 ', '$13 ', '$152 ', '$105 ', '$90 ',
       '$186 ', '$66 ', '$225 ', '$19 ', '$227 ', '$32 ', '$189 ',
       '$101 ', '$23 ', '$77 ', '$190 ', '$75 ', '$231 ', '$133 ', '$35 ',
       '$82 ', '$120 ', '$21 ', '$145 ', '$65 ', '$26 ', '$89 ', '$218 ',
       '$104 ', '$44 ', '$63 ', '$79 ', '$163 ', '$22 ', '$182 ', '$165 ',
       '$37 ', '$17 ', '$48 ', '$11 ', '$131 ', '$102 ', '$240 ', '$119 ',
       '$215 ', '$191 ', '$16 ', '$233 ', '$136 ', '$170 ', '$201 ',
       '$109 ', '$123 ', '$36 ', '$12 ', '$180 ', '$238 ', '$157 ',
       '$14 ', '$198 ', '$206 ', '$122 ', '$208 ', '$140 ', '$10 ',
       '$20 ', '$110 ', '$88 ', '$98 ', '$234 ', '$151 ', '$172 ',
       '$107 ', '$138 ', '$194 ', '$139 ', '$117 ', '$31 ', '$156 ',
       '$124 ', '$175 ', '$162 ', '$187 ', '$93 ', '$51 ', '$143 ',
       '$239 '

In [147]:
df['SERVICE FEE'] = df['SERVICE FEE'].str.split('$').str[1]

In [148]:
df['SERVICE FEE'] = df['SERVICE FEE'].astype(int)

In [149]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 98509 entries, 0 to 102040
Data columns (total 22 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   ID                              98509 non-null  int64  
 1   NAME                            98509 non-null  object 
 2   HOST ID                         98509 non-null  int64  
 3   HOST_IDENTITY_VERIFIED          98509 non-null  object 
 4   HOST NAME                       98509 non-null  object 
 5   NEIGHBOURHOOD GROUP             98509 non-null  object 
 6   NEIGHBOURHOOD                   98509 non-null  object 
 7   LAT                             98509 non-null  float64
 8   LONG                            98509 non-null  float64
 9   COUNTRY                         98509 non-null  object 
 10  COUNTRY CODE                    98509 non-null  object 
 11  INSTANT_BOOKABLE                98509 non-null  object 
 12  CANCELLATION_POLICY             9850

# PRICE

In [150]:
df['PRICE']

0           $966 
1           $142 
4           $204 
5           $577 
7         $1,060 
           ...   
102029      $822 
102030      $455 
102031    $1,078 
102032      $103 
102040      $982 
Name: PRICE, Length: 98509, dtype: object

In [151]:
df['PRICE'].unique()

array(['$966 ', '$142 ', '$204 ', ..., '$917 ', '$309 ', '$770 '],
      dtype=object)

In [152]:
df['PRICE'] = df['PRICE'].str.split('$').str[1].str.replace(',','')

In [153]:
df['PRICE'] = df['PRICE'].astype(int)

In [154]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 98509 entries, 0 to 102040
Data columns (total 22 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   ID                              98509 non-null  int64  
 1   NAME                            98509 non-null  object 
 2   HOST ID                         98509 non-null  int64  
 3   HOST_IDENTITY_VERIFIED          98509 non-null  object 
 4   HOST NAME                       98509 non-null  object 
 5   NEIGHBOURHOOD GROUP             98509 non-null  object 
 6   NEIGHBOURHOOD                   98509 non-null  object 
 7   LAT                             98509 non-null  float64
 8   LONG                            98509 non-null  float64
 9   COUNTRY                         98509 non-null  object 
 10  COUNTRY CODE                    98509 non-null  object 
 11  INSTANT_BOOKABLE                98509 non-null  object 
 12  CANCELLATION_POLICY             9850

# ROOM TYPE

In [155]:
df['ROOM TYPE']

0            Private room
1         Entire home/apt
4         Entire home/apt
5         Entire home/apt
7            Private room
               ...       
102029       Private room
102030       Private room
102031       Private room
102032       Private room
102040       Private room
Name: ROOM TYPE, Length: 98509, dtype: object

In [156]:
df['ROOM TYPE'].unique()

array(['Private room', 'Entire home/apt', 'Shared room', 'Hotel room'],
      dtype=object)

In [157]:
from sklearn.preprocessing import OneHotEncoder

#encoder = OneHotEncoder()

In [158]:
#encoded = encoder.fit_transform(df[['ROOM TYPE']]).toarray()

In [159]:
#df_room_type = pd.DataFrame(encoded,columns=encoder.get_feature_names_out())

In [160]:
#df_room_type

In [161]:
#df = pd.concat([df,df_room_type],axis=1)

In [162]:
#df.drop(columns=['ROOM TYPE'],inplace=True)

In [163]:
#df.head(3)

In [164]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 98509 entries, 0 to 102040
Data columns (total 22 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   ID                              98509 non-null  int64  
 1   NAME                            98509 non-null  object 
 2   HOST ID                         98509 non-null  int64  
 3   HOST_IDENTITY_VERIFIED          98509 non-null  object 
 4   HOST NAME                       98509 non-null  object 
 5   NEIGHBOURHOOD GROUP             98509 non-null  object 
 6   NEIGHBOURHOOD                   98509 non-null  object 
 7   LAT                             98509 non-null  float64
 8   LONG                            98509 non-null  float64
 9   COUNTRY                         98509 non-null  object 
 10  COUNTRY CODE                    98509 non-null  object 
 11  INSTANT_BOOKABLE                98509 non-null  object 
 12  CANCELLATION_POLICY             9850

# CANCELLATION_POLICY

In [165]:
df['CANCELLATION_POLICY']

0           strict
1         moderate
4         moderate
5         flexible
7         moderate
            ...   
102029    flexible
102030    moderate
102031    flexible
102032    moderate
102040      strict
Name: CANCELLATION_POLICY, Length: 98509, dtype: object

In [166]:
df['CANCELLATION_POLICY'].unique()

array(['strict', 'moderate', 'flexible'], dtype=object)

In [167]:
df['CANCELLATION_POLICY'].isna().sum()

0

In [168]:
df[df['CANCELLATION_POLICY'].isna()]

,ID,NAME,HOST ID,HOST_IDENTITY_VERIFIED,HOST NAME,NEIGHBOURHOOD GROUP,NEIGHBOURHOOD,LAT,LONG,COUNTRY,...,CANCELLATION_POLICY,ROOM TYPE,CONSTRUCTION YEAR,PRICE,SERVICE FEE,MINIMUM NIGHTS,NUMBER OF REVIEWS,REVIEW RATE NUMBER,CALCULATED HOST LISTINGS COUNT,AVAILABILITY 365


In [169]:
df.dropna(inplace=True)

In [170]:
df['CANCELLATION_POLICY'].unique()

array(['strict', 'moderate', 'flexible'], dtype=object)

In [171]:
#encoded = encoder.fit_transform(df[['CANCELLATION_POLICY']]).toarray()

In [172]:
#encoded

In [173]:
#df_can_policy = pd.DataFrame(encoded,columns=encoder.get_feature_names_out())

In [174]:
#df_can_policy

In [175]:
#df = pd.concat([df,df_can_policy],axis=True)

In [176]:
#df.drop(columns=['CANCELLATION_POLICY'],inplace=True)

In [177]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 98509 entries, 0 to 102040
Data columns (total 22 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   ID                              98509 non-null  int64  
 1   NAME                            98509 non-null  object 
 2   HOST ID                         98509 non-null  int64  
 3   HOST_IDENTITY_VERIFIED          98509 non-null  object 
 4   HOST NAME                       98509 non-null  object 
 5   NEIGHBOURHOOD GROUP             98509 non-null  object 
 6   NEIGHBOURHOOD                   98509 non-null  object 
 7   LAT                             98509 non-null  float64
 8   LONG                            98509 non-null  float64
 9   COUNTRY                         98509 non-null  object 
 10  COUNTRY CODE                    98509 non-null  object 
 11  INSTANT_BOOKABLE                98509 non-null  object 
 12  CANCELLATION_POLICY             9850

# INSTANT_BOOKABLE

In [178]:
df['INSTANT_BOOKABLE']

0         False
1         False
4         False
5          True
7         False
          ...  
102029    False
102030     True
102031     True
102032    False
102040     True
Name: INSTANT_BOOKABLE, Length: 98509, dtype: object

In [179]:
df['INSTANT_BOOKABLE'].unique()

array([False, True], dtype=object)

In [180]:
df.dropna(inplace=True)

In [181]:
df['INSTANT_BOOKABLE'].unique()

array([False, True], dtype=object)

In [182]:
df['INSTANT_BOOKABLE'].isna().sum()

0

In [183]:
df['INSTANT_BOOKABLE'] = df['INSTANT_BOOKABLE'].apply(lambda x:1 if x==True else 0)

In [184]:
df['INSTANT_BOOKABLE'].value_counts()

INSTANT_BOOKABLE
0    49485
1    49024
Name: count, dtype: int64

In [185]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 98509 entries, 0 to 102040
Data columns (total 22 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   ID                              98509 non-null  int64  
 1   NAME                            98509 non-null  object 
 2   HOST ID                         98509 non-null  int64  
 3   HOST_IDENTITY_VERIFIED          98509 non-null  object 
 4   HOST NAME                       98509 non-null  object 
 5   NEIGHBOURHOOD GROUP             98509 non-null  object 
 6   NEIGHBOURHOOD                   98509 non-null  object 
 7   LAT                             98509 non-null  float64
 8   LONG                            98509 non-null  float64
 9   COUNTRY                         98509 non-null  object 
 10  COUNTRY CODE                    98509 non-null  object 
 11  INSTANT_BOOKABLE                98509 non-null  int64  
 12  CANCELLATION_POLICY             9850

# COUNTRY & COUNTRY CODE

In [186]:
df['COUNTRY CODE']

0         US
1         US
4         US
5         US
7         US
          ..
102029    US
102030    US
102031    US
102032    US
102040    US
Name: COUNTRY CODE, Length: 98509, dtype: object

In [187]:
df['COUNTRY CODE'].value_counts()

COUNTRY CODE
US    98509
Name: count, dtype: int64

In [188]:
df['COUNTRY'].value_counts()

COUNTRY
United States    98509
Name: count, dtype: int64

In [189]:
df['COUNTRY'] = 1
df['COUNTRY CODE'] = 1

In [190]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 98509 entries, 0 to 102040
Data columns (total 22 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   ID                              98509 non-null  int64  
 1   NAME                            98509 non-null  object 
 2   HOST ID                         98509 non-null  int64  
 3   HOST_IDENTITY_VERIFIED          98509 non-null  object 
 4   HOST NAME                       98509 non-null  object 
 5   NEIGHBOURHOOD GROUP             98509 non-null  object 
 6   NEIGHBOURHOOD                   98509 non-null  object 
 7   LAT                             98509 non-null  float64
 8   LONG                            98509 non-null  float64
 9   COUNTRY                         98509 non-null  int64  
 10  COUNTRY CODE                    98509 non-null  int64  
 11  INSTANT_BOOKABLE                98509 non-null  int64  
 12  CANCELLATION_POLICY             9850

# NEIGHBOURHOOD

In [191]:
df['NEIGHBOURHOOD']

0                 Kensington
1                    Midtown
4                East Harlem
5                Murray Hill
7         Bedford-Stuyvesant
                 ...        
102029               Astoria
102030    Bedford-Stuyvesant
102031          Williamsburg
102032               Bayside
102040         East Elmhurst
Name: NEIGHBOURHOOD, Length: 98509, dtype: object

In [192]:
df['NEIGHBOURHOOD'].unique()

array(['Kensington', 'Midtown', 'East Harlem', 'Murray Hill',
       'Bedford-Stuyvesant', "Hell's Kitchen", 'Upper West Side',
       'Chinatown', 'South Slope', 'Bushwick', 'Flatbush', 'East Village',
       'Williamsburg', 'Harlem', 'Gowanus', 'Clinton Hill', 'Greenpoint',
       'Flatlands', 'Fort Greene', 'West Village', 'Prospect Heights',
       'DUMBO', 'Upper East Side', 'Financial District', 'Ridgewood',
       'Morningside Heights', 'Park Slope', 'Ditmars Steinway',
       'Cobble Hill', 'Flatiron District', 'Windsor Terrace',
       'Roosevelt Island', 'Lower East Side', 'Chelsea',
       'Carroll Gardens', 'Little Italy', 'Inwood', 'Kingsbridge',
       'Astoria', 'Boerum Hill', 'Washington Heights', 'Rockaway Beach',
       'Crown Heights', 'Nolita', 'Woodlawn', 'Sunnyside', 'Gramercy',
       'East New York', 'Prospect-Lefferts Gardens', 'Concourse Village',
       'Sheepshead Bay', 'Theater District', 'Emerson Hill',
       'Long Island City', 'Fort Hamilton', 'Bensonhu

In [193]:
df['NEIGHBOURHOOD'].isna().sum()

0

In [194]:
from sklearn.preprocessing import LabelEncoder

#label = LabelEncoder()

In [195]:
#encoded = label.fit_transform(df[['NEIGHBOURHOOD']])

In [196]:
#encoded

In [197]:
'''df_neighbourhood = pd.DataFrame({
    'NEIGHBOURHOOD_ENCODED' : encoded
})'''

"df_neighbourhood = pd.DataFrame({\n    'NEIGHBOURHOOD_ENCODED' : encoded\n})"

In [198]:
#df_neighbourhood

In [199]:
#df = pd.concat([df,df_neighbourhood],axis=1)

In [200]:
#df.drop(columns=['NEIGHBOURHOOD'],inplace=True)

In [201]:
df.dropna(inplace=True)

In [202]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 98509 entries, 0 to 102040
Data columns (total 22 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   ID                              98509 non-null  int64  
 1   NAME                            98509 non-null  object 
 2   HOST ID                         98509 non-null  int64  
 3   HOST_IDENTITY_VERIFIED          98509 non-null  object 
 4   HOST NAME                       98509 non-null  object 
 5   NEIGHBOURHOOD GROUP             98509 non-null  object 
 6   NEIGHBOURHOOD                   98509 non-null  object 
 7   LAT                             98509 non-null  float64
 8   LONG                            98509 non-null  float64
 9   COUNTRY                         98509 non-null  int64  
 10  COUNTRY CODE                    98509 non-null  int64  
 11  INSTANT_BOOKABLE                98509 non-null  int64  
 12  CANCELLATION_POLICY             9850

# NEIGHBOURHOOD GROUP

In [203]:
df['NEIGHBOURHOOD GROUP']

0          Brooklyn
1         Manhattan
4         Manhattan
5         Manhattan
7          Brooklyn
            ...    
102029       Queens
102030     Brooklyn
102031     Brooklyn
102032       Queens
102040       Queens
Name: NEIGHBOURHOOD GROUP, Length: 98509, dtype: object

In [204]:
df['NEIGHBOURHOOD GROUP'].isna().sum()

0

In [205]:
df['NEIGHBOURHOOD GROUP'].unique()

array(['Brooklyn', 'Manhattan', 'brookln', 'Queens', 'Bronx',
       'Staten Island'], dtype=object)

In [206]:
#encoded = label.fit_transform(df[['NEIGHBOURHOOD GROUP']])

In [207]:
#encoded

In [208]:
''' = pd.DataFrame({
    'NEIGHBOURHOOD GROUP_ENCODED' : encoded
})'''

" = pd.DataFrame({\n    'NEIGHBOURHOOD GROUP_ENCODED' : encoded\n})"

In [209]:
#df_neighbourhood_group

In [210]:
#df = pd.concat([df,df_neighbourhood_group],axis=1)

In [211]:
#df.drop(columns=['NEIGHBOURHOOD GROUP'],inplace=True)

In [212]:
df.dropna(inplace=True)

In [213]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 98509 entries, 0 to 102040
Data columns (total 22 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   ID                              98509 non-null  int64  
 1   NAME                            98509 non-null  object 
 2   HOST ID                         98509 non-null  int64  
 3   HOST_IDENTITY_VERIFIED          98509 non-null  object 
 4   HOST NAME                       98509 non-null  object 
 5   NEIGHBOURHOOD GROUP             98509 non-null  object 
 6   NEIGHBOURHOOD                   98509 non-null  object 
 7   LAT                             98509 non-null  float64
 8   LONG                            98509 non-null  float64
 9   COUNTRY                         98509 non-null  int64  
 10  COUNTRY CODE                    98509 non-null  int64  
 11  INSTANT_BOOKABLE                98509 non-null  int64  
 12  CANCELLATION_POLICY             9850

In [214]:
df['HOST NAME']

0          Madaline
1             Jenna
4            Lyndon
5          Michelle
7              Emma
            ...    
102029     Cristina
102030        Petra
102031      Russell
102032    Christine
102040        Sonia
Name: HOST NAME, Length: 98509, dtype: object

In [215]:
df['HOST NAME'].unique()

array(['Madaline', 'Jenna', 'Lyndon', ..., 'Minh', 'Dier', 'Apostle John'],
      dtype=object)

In [216]:
#encoded = label.fit_transform(df[['HOST NAME']])

In [217]:
#encoded

In [218]:
#df_hostname = pd.DataFrame({
#})

In [219]:
#df_hostname

In [220]:
#df = pd.concat([df,df_hostname],axis=1)

In [221]:
#df.drop(columns=['HOST NAME'],inplace=True)

In [222]:
#df.dropna(inplace=True)

In [223]:
#df.head()

In [224]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 98509 entries, 0 to 102040
Data columns (total 22 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   ID                              98509 non-null  int64  
 1   NAME                            98509 non-null  object 
 2   HOST ID                         98509 non-null  int64  
 3   HOST_IDENTITY_VERIFIED          98509 non-null  object 
 4   HOST NAME                       98509 non-null  object 
 5   NEIGHBOURHOOD GROUP             98509 non-null  object 
 6   NEIGHBOURHOOD                   98509 non-null  object 
 7   LAT                             98509 non-null  float64
 8   LONG                            98509 non-null  float64
 9   COUNTRY                         98509 non-null  int64  
 10  COUNTRY CODE                    98509 non-null  int64  
 11  INSTANT_BOOKABLE                98509 non-null  int64  
 12  CANCELLATION_POLICY             9850

# HOST_IDENTITY_VERIFIED

In [225]:
df['HOST_IDENTITY_VERIFIED']

0         unconfirmed
1            verified
4            verified
5            verified
7         unconfirmed
             ...     
102029    unconfirmed
102030       verified
102031       verified
102032    unconfirmed
102040       verified
Name: HOST_IDENTITY_VERIFIED, Length: 98509, dtype: object

In [226]:
df['HOST_IDENTITY_VERIFIED'].unique()

array(['unconfirmed', 'verified'], dtype=object)

In [227]:
df['HOST_IDENTITY_VERIFIED'].isna().sum()

0

In [228]:
df['HOST_IDENTITY_VERIFIED'] = df['HOST_IDENTITY_VERIFIED'].apply(lambda x:1 if x=='verified' else 0)

In [229]:
df['HOST_IDENTITY_VERIFIED'].value_counts()

HOST_IDENTITY_VERIFIED
0    49283
1    49226
Name: count, dtype: int64

In [230]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 98509 entries, 0 to 102040
Data columns (total 22 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   ID                              98509 non-null  int64  
 1   NAME                            98509 non-null  object 
 2   HOST ID                         98509 non-null  int64  
 3   HOST_IDENTITY_VERIFIED          98509 non-null  int64  
 4   HOST NAME                       98509 non-null  object 
 5   NEIGHBOURHOOD GROUP             98509 non-null  object 
 6   NEIGHBOURHOOD                   98509 non-null  object 
 7   LAT                             98509 non-null  float64
 8   LONG                            98509 non-null  float64
 9   COUNTRY                         98509 non-null  int64  
 10  COUNTRY CODE                    98509 non-null  int64  
 11  INSTANT_BOOKABLE                98509 non-null  int64  
 12  CANCELLATION_POLICY             9850

# NAME

In [231]:
df['NAME']

0                       Clean & quiet apt home by the park
1                                    Skylit Midtown Castle
4         Entire Apt: Spacious Studio/Loft by central park
5                Large Cozy 1 BR Apartment In Midtown East
7                                          BlissArtsSpace!
                                ...                       
102029              Cozy room in a 2B with backyard access
102030                        An East Williamsburg Delight
102031                        Cozy bedroom in williamsburg
102032                              Spacious Room w/ 2 Bed
102040                       Room in Queens, NY, near LGA.
Name: NAME, Length: 98509, dtype: object

In [232]:
df['NAME'].unique()

array(['Clean & quiet apt home by the park', 'Skylit Midtown Castle',
       'Entire Apt: Spacious Studio/Loft by central park', ...,
       'Elmhurst 1st Fl Rightl Bedroom',
       'Elmhurst 1st Floor BR w/ Pvt. Bath&Balcony',
       'Parisian Style Apartment in Heart of Brooklyn'], dtype=object)

In [233]:
df['NAME'].value_counts()

NAME
Home away from home                                  32
Brooklyn Apartment                                   27
New york Multi-unit building                         27
Water View King Bed Hotel Room                       25
Private Room                                         23
                                                     ..
Minimal Loft in heart of Williamsburg -waterfront     1
Spacious Home with Backyard in Brooklyn               1
Entire Apartment in Beautiful Brooklyn Brownstone     1
Entire floor of apartment with separate entrance      1
Parisian Style Apartment in Heart of Brooklyn         1
Name: count, Length: 59611, dtype: int64

In [234]:
#encoded = label.fit_transform(df[['NAME']])

In [235]:
#encoded

In [236]:
#df_name = pd.DataFrame({
#    'NAME_ENCODED' : encoded
#})

In [237]:
#df_name

In [238]:
#df = pd.concat([df,df_name],axis=1)

In [239]:
#df.drop(columns=['NAME'],inplace=True)

In [240]:
#df.dropna(inplace=True)

In [241]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 98509 entries, 0 to 102040
Data columns (total 22 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   ID                              98509 non-null  int64  
 1   NAME                            98509 non-null  object 
 2   HOST ID                         98509 non-null  int64  
 3   HOST_IDENTITY_VERIFIED          98509 non-null  int64  
 4   HOST NAME                       98509 non-null  object 
 5   NEIGHBOURHOOD GROUP             98509 non-null  object 
 6   NEIGHBOURHOOD                   98509 non-null  object 
 7   LAT                             98509 non-null  float64
 8   LONG                            98509 non-null  float64
 9   COUNTRY                         98509 non-null  int64  
 10  COUNTRY CODE                    98509 non-null  int64  
 11  INSTANT_BOOKABLE                98509 non-null  int64  
 12  CANCELLATION_POLICY             9850

In [242]:
df.rename(columns={
    "ID": "id"
}, inplace=True)

In [243]:
df.duplicated().sum()

0

In [244]:
df.head()

,id,NAME,HOST ID,HOST_IDENTITY_VERIFIED,HOST NAME,NEIGHBOURHOOD GROUP,NEIGHBOURHOOD,LAT,LONG,COUNTRY,...,CANCELLATION_POLICY,ROOM TYPE,CONSTRUCTION YEAR,PRICE,SERVICE FEE,MINIMUM NIGHTS,NUMBER OF REVIEWS,REVIEW RATE NUMBER,CALCULATED HOST LISTINGS COUNT,AVAILABILITY 365
0,1001254,Clean & quiet apt home by the park,80014485718,0,Madaline,Brooklyn,Kensington,40.64749,-73.97237,1,...,strict,Private room,2020.0,966,193,10.0,9.0,4.0,6.0,286.0
1,1002102,Skylit Midtown Castle,52335172823,1,Jenna,Manhattan,Midtown,40.75362,-73.98377,1,...,moderate,Entire home/apt,2007.0,142,28,30.0,45.0,4.0,2.0,228.0
4,1003689,Entire Apt: Spacious Studio/Loft by central park,92037596077,1,Lyndon,Manhattan,East Harlem,40.79851,-73.94399,1,...,moderate,Entire home/apt,2009.0,204,41,10.0,9.0,3.0,1.0,289.0
5,1004098,Large Cozy 1 BR Apartment In Midtown East,45498551794,1,Michelle,Manhattan,Murray Hill,40.74767,-73.97500,1,...,flexible,Entire home/apt,2013.0,577,115,3.0,74.0,3.0,1.0,374.0
7,1005202,BlissArtsSpace!,90821839709,0,Emma,Brooklyn,Bedford-Stuyvesant,40.68688,-73.95596,1,...,moderate,Private room,2009.0,1060,212,45.0,49.0,5.0,1.0,219.0


In [245]:
# prompt: create a new csv file with updated datatha has new data

# Assuming df is the DataFrame from the provided code
df.to_csv('updated_airbnb_data.csv', index=False)


# CREATING THE airbnb_nyc SCHEMA

In [246]:
# Step 1: Connect without specifying the database
conn = mysql.connector.connect(
    host="127.0.0.1",
    user="root",
    password="Bonakele.1"
)
cursor = conn.cursor()

# Step 2: Create the database if it doesn't exist
cursor.execute("CREATE DATABASE IF NOT EXISTS airbnb_nyc")
cursor.execute("USE airbnb_nyc")

# CREATE TABLES IN SCHEMA
- Come up with table names
- Insert values for tables with unique and few values 
- Upload cleaned airbnb_nyc for table seperation
- Standardize column names

In [247]:
# Neighbourhood Groups
cursor.execute("""
CREATE TABLE IF NOT EXISTS neighbourhood_groups (
    id INT AUTO_INCREMENT PRIMARY KEY,
    name VARCHAR(50) NOT NULL UNIQUE
)
""")

# Neighbourhoods
cursor.execute("""
CREATE TABLE IF NOT EXISTS neighbourhoods (
    id INT AUTO_INCREMENT PRIMARY KEY,
    name VARCHAR(100) NOT NULL,
    neighbourhood_group_id INT NOT NULL,
    UNIQUE (name, neighbourhood_group_id),
    FOREIGN KEY (neighbourhood_group_id) REFERENCES neighbourhood_groups(id)
)
""")

# Room Types
cursor.execute("""
CREATE TABLE IF NOT EXISTS room_types (
    id INT AUTO_INCREMENT PRIMARY KEY,
    type VARCHAR(50) NOT NULL UNIQUE
)
""")

# Cancellation Policies
cursor.execute("""
CREATE TABLE IF NOT EXISTS cancellation_policies (
    id INT AUTO_INCREMENT PRIMARY KEY,
    policy VARCHAR(50) NOT NULL UNIQUE
)
""")

# Hosts
cursor.execute("""
CREATE TABLE IF NOT EXISTS hosts (
    id BIGINT PRIMARY KEY,
    name VARCHAR(100),
    identity_verified VARCHAR(20)
)
""")

# Listings
cursor.execute("""
CREATE TABLE IF NOT EXISTS listings (
    id BIGINT PRIMARY KEY,
    name VARCHAR(255),
    host_id BIGINT NOT NULL,
    neighbourhood_id INT,
    latitude DECIMAL(10, 8),
    longitude DECIMAL(11, 8),
    country VARCHAR(50),
    country_code VARCHAR(10),
    room_type_id INT,
    construction_year INT,
    price DECIMAL(10, 2),
    service_fee DECIMAL(10, 2),
    minimum_nights INT,
    number_of_reviews INT,
    last_review DATE,
    reviews_per_month DECIMAL(5, 2),
    review_rate_number INT,
    calculated_host_listings_count INT,
    availability_365 INT,
    instant_bookable BOOLEAN,
    cancellation_policy_id INT,
    house_rules TEXT,
    license TEXT,
    FOREIGN KEY (host_id) REFERENCES hosts(id),
    FOREIGN KEY (neighbourhood_id) REFERENCES neighbourhoods(id),
    FOREIGN KEY (room_type_id) REFERENCES room_types(id),
    FOREIGN KEY (cancellation_policy_id) REFERENCES cancellation_policies(id)
)
""")

cursor.execute("INSERT IGNORE INTO neighbourhood_groups (name) VALUES ('Brooklyn'), ('Manhattan'), ('Queens'), ('Bronx'), ('Staten Island')")
cursor.execute("INSERT IGNORE INTO room_types (type) VALUES ('Private room'), ('Entire home/apt'), ('Shared room')")
cursor.execute("INSERT IGNORE INTO cancellation_policies (policy) VALUES ('strict'), ('moderate'), ('flexible')")
conn.commit()

df = pd.read_csv("updated_airbnb_data.csv")

# Standardize column names
df.columns = [col.strip().lower().replace(" ", "_") for col in df.columns]

# INSERT VALUES FOR TABLES WITH A LOT OF ROWS USING DF

In [249]:
# Insert unique neighbourhoods
neighbourhoods = df[['neighbourhood', 'neighbourhood_group']].drop_duplicates()

for _, row in neighbourhoods.iterrows():
    group_name = row['neighbourhood_group']
    if pd.notnull(group_name):  # Skip if it's NaN
        group_name = str(group_name)  # Convert to string just in case
        cursor.execute("SELECT id FROM neighbourhood_groups WHERE name = %s", (group_name,))
        group_id = cursor.fetchone()
        if group_id:
            # Now insert into neighbourhoods table
            cursor.execute("""
                INSERT IGNORE INTO neighbourhoods (name, neighbourhood_group_id)
                VALUES (%s, %s)
            """, (row['neighbourhood'], group_id[0]))

# Insert unique hosts  

hosts = df[['host_id', 'host_name', 'host_identity_verified']].drop_duplicates()

# Ensure types are correct
hosts['host_id'] = hosts['host_id'].astype(int)
hosts['host_name'] = hosts['host_name'].astype(str)
hosts['host_identity_verified'] = hosts['host_identity_verified'].astype(str)

# Now insert
for _, row in hosts.iterrows():
    cursor.execute("""
        INSERT IGNORE INTO hosts (id, name, identity_verified)
        VALUES (%s, %s, %s)
    """, (row['host_id'], row['host_name'], row['host_identity_verified']))



# Insert unique listings
df_clean = df.dropna(subset=['neighbourhood', 'room_type', 'cancellation_policy', 'host_id'])

# Ensure all lookup columns are the correct types
df_clean['neighbourhood'] = df_clean['neighbourhood'].astype(str)
df_clean['room_type'] = df_clean['room_type'].astype(str)
df_clean['cancellation_policy'] = df_clean['cancellation_policy'].astype(str)
df_clean['host_id'] = df_clean['host_id'].astype(int)

# Then insert listings (or perform the lookup)
for _, row in df_clean.iterrows():
    cursor.execute("SELECT id FROM neighbourhoods WHERE name = %s", (row['neighbourhood'],))
    neighbourhood_id = cursor.fetchone()

    cursor.execute("SELECT id FROM room_types WHERE type = %s", (row['room_type'],))
    room_type_id = cursor.fetchone()

    cursor.execute("SELECT id FROM cancellation_policies WHERE policy = %s", (row['cancellation_policy'],))
    cancellation_policy_id = cursor.fetchone()
    
    # Insert listing
    cursor.execute("""
        INSERT IGNORE INTO listings (
            id, name, host_id, neighbourhood_id, latitude, longitude, country,
            country_code, room_type_id, construction_year, price, service_fee,
            minimum_nights, number_of_reviews, last_review, reviews_per_month,
            review_rate_number, calculated_host_listings_count, availability_365,
            instant_bookable, cancellation_policy_id, house_rules, license
        )
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """, (
        row['id'], row['name'], row['host_id'], neighbourhood_id[0] if neighbourhood_id else None,
        row['lat'], row['long'], row['country'], row['country_code'], room_type_id[0] if room_type_id else None,
        row['construction_year'], row['price'], row['service_fee'],
        row['minimum_nights'], row['number_of_reviews'], row['last_review'],
        row['reviews_per_month'], row['review_rate_number'],
        row['calculated_host_listings_count'], row['availability_365'],
        row['instant_bookable'], cancellation_policy_id[0] if cancellation_policy_id else None,
        row['house_rules'], row['license']
    ))

    conn.commit()

KeyboardInterrupt: 

In [ ]:
# run this on anaconda prompt
## mysqldump -u root -p airbnb_nyc > airbnb_nyc.sql
# If this do not work run this To save a sql file:
## "C:\Program Files\MySQL\MySQL Server 8.0\bin\mysqldump.exe" -u root -p airbnb_nyc > airbnb_nyc.sql

# CREATE DATABASE FILE

In [ ]:
# Connect to MySQL
mysql_conn = mysql.connector.connect(
    host="127.0.0.1",
    user="root",
    password="Bonakele.1",
    database="airbnb_nyc"
)
mysql_cursor = mysql_conn.cursor()

# Connect to SQLite (creates airbnb_nyc.db if it doesn't exist)
sqlite_conn = sqlite3.connect("airbnb_nyc.db")

# List of table names to transfer
tables = [
    "neighbourhood_groups",
    "neighbourhoods",
    "room_types",
    "cancellation_policies",
    "hosts",
    "listings"
]

# Transfer each table from MySQL to SQLite
for table in tables:
    df = pd.read_sql(f"SELECT * FROM {table}", mysql_conn)
    df.to_sql(table, sqlite_conn, if_exists="replace", index=False)
    print(f"Transferred table: {table}")

# Close connections
mysql_cursor.close()
mysql_conn.close()
sqlite_conn.close()

In [ ]:
# Load your database and create a database connection.
# You can connect to the sql database in any way you wish. 
# Use this method if you are unsure how to proceed. 
# Ensure the bike_store.db file is in the same directory as this notebook.
try:
    with sqlite3.connect("airbnb_nyc.db") as conn:
        print(f"Opened SQLite database with version {sqlite3.sqlite_version} successfully.")

except sqlite3.OperationalError as e:
    print("Failed to open database:", e)

# List all tables in the database
pd.read_sql('''SELECT name FROM sqlite_master WHERE type='table';''',conn)     #Does the same thing